# **Damaris Campos PHW7 Ch 7: Beyond the Fundamentals of Querying**

Tables created in order to solve the propositions from Ch.7:

In [83]:
USE TSQLV6
DROP TABLE IF EXISTS dbo.Orders;
CREATE TABLE dbo.Orders
(
orderid INT NOT NULL,
orderdate DATE NOT NULL,
empid INT NOT NULL,
custid VARCHAR(5) NOT NULL,
qty INT NOT NULL,
CONSTRAINT PK_Orders PRIMARY KEY(orderid)
);
INSERT INTO dbo.Orders(orderid, orderdate, empid, custid, qty)
VALUES
(30001, '20140802', 3, 'A', 10),
(10001, '20141224', 2, 'A', 12),
(10005, '20141224', 1, 'B', 20),
(40001, '20150109', 2, 'A', 40),
(10006, '20150118', 1, 'C', 14),
(20001, '20150212', 2, 'B', 12),
(40005, '20160212', 3, 'A', 10),
(20002, '20160216', 1, 'C', 20),
(30003, '20160418', 2, 'B', 15),
(30004, '20140418', 3, 'C', 22),
(30007, '20160907', 3, 'D', 30);



(11 rows affected)

Total execution time: 00:00:00.018

In [38]:
USE Northwinds2022TSQLV7
DROP TABLE IF EXISTS dbo.Orders;
CREATE TABLE dbo.Orders
(
OrderId INT NOT NULL,
OrderDate DATE NOT NULL,
EmployeeId INT NOT NULL,
CustomerId VARCHAR(5) NOT NULL,
Quantity INT NOT NULL,
CONSTRAINT PK_Orders PRIMARY KEY(orderid)
);
INSERT INTO dbo.Orders(OrderId, OrderDate, EmployeeId, CustomerId, Quantity)
VALUES
(30001, '20140802', 3, 'A', 10),
(10001, '20141224', 2, 'A', 12),
(10005, '20141224', 1, 'B', 20),
(40001, '20150109', 2, 'A', 40),
(10006, '20150118', 1, 'C', 14),
(20001, '20150212', 2, 'B', 12),
(40005, '20160212', 3, 'A', 10),
(20002, '20160216', 1, 'C', 20),
(30003, '20160418', 2, 'B', 15),
(30004, '20140418', 3, 'C', 22),
(30007, '20160907', 3, 'D', 30);


--DROP TABLE IF EXISTS dbo.Orders;


(11 rows affected)

Total execution time: 00:00:00.037

# **<mark>Top 5 Queries</mark>**

## TOP 1:

**Proposition**

- Write a query against the dbo.Orders table that computes for each customer order:
    - the difference between the current order quantity and the customer's previous order quantity
    - the difference between the current order quantity and the customer's next order quantity.

**Desired Output:**

| custid | orderid | qty | diffprev | diffnext |
| --- | --- | --- | --- | --- |
| A | 30001 | 10 | NULL | \-2 |
| A | 10001 | 12 | 2 | \-28 |
| A | 40001 | 40 | 28 | 30 |
| A | 40005 | 10 | \-30 | NULL |
| B | 10005 | 20 | NULL | 8 |
| B | 20001 | 12 | \-8 | \-3 |
| B | 30003 | 15 | 3 | NULL |
| C | 30004 | 22 | NULL | 8 |
| C | 10006 | 14 | \-8 | \-6 |
| C | 20002 | 20 | 6 | NULL |
| D | 30007 | 30 | NULL | NULL |

In [52]:
/*
USE TSQLV4
SELECT 
    custid, 
    orderid, 
    qty,
    qty - LAG(qty) OVER(PARTITION BY custid ORDER BY orderdate, orderid) AS diffprev,
    qty - LEAD(qty) OVER(PARTITION BY custid ORDER BY orderdate, orderid) AS diffnext
FROM 
    dbo.Orders;
*/
USE Northwinds2022TSQLV7
SELECT 
    CustomerId, 
    OrderId, 
    Quantity,
    Quantity - LAG(Quantity) OVER(PARTITION BY CustomerId ORDER BY OrderDate, OrderId) AS DiffPrev,
    Quantity - LEAD(Quantity) OVER(PARTITION BY CustomerId ORDER BY OrderDate, OrderId) AS DiffNext
FROM 
    dbo.Orders;

(11 rows affected)

Total execution time: 00:00:00.006

CustomerId,OrderId,Quantity,DiffPrev,DiffNext
A,30001,10,NULL,-2
A,10001,12,2,-28
A,40001,40,28,30
A,40005,10,-30,NULL
B,10005,20,NULL,8
B,20001,12,-8,-3
B,30003,15,3,NULL
C,30004,22,NULL,8
C,10006,14,-8,-6
C,20002,20,6,NULL


**Explanation**

- The <u>proposition</u> is to generate a query against dbo.Orders table that calculates for each customer order, the difference between the current order quantity and both the previous and next order quantities for that customer. <u>Table</u>s used are dbo.Orders containing the <u>columns</u> CustomerId, OrderId, Quantity, OrderDate. The <u>predicate:</u>
    - 'LAG(Quantity) OVER(PARTITION BY CustomerId ORDER BY OrderDate, OrderId) AS DiffPrev', this calculates the difference between the current order quantity and the previous order quantity for each customer, considering the order date and order id for the ordering. 
    - 'LEAD(Quantity) OVER(PARTITION BY CustomerId ORDER BY OrderDate, OrderId) AS DiffNext' computes the difference between the current order quantity and the next order quantity for each customer, based on the order date and order ID for ordering. 
    - OVER(PARTITION BY CustomerId ORDER BY OrderDate, OrderId) clause partitions the data by CustomerId and then orders it by OrderDate and OrderId.  
- <span style="color: var(--vscode-foreground);">This ensures that the lag and lead functions operate on the data for each customer in the correct order based on the order date and order ID. The computed differences are returned as DiffPrev and DiffNext in the result set along with the CustomerId and OrderId for each order.</span>

## _TOP 2_

**Proposition**

- Write a query against the dbo.Orders table that calculates the total quantity of products ordered, considering various grouping combinations such as by EmployeeId, CustomerId, both EmployeeId and CustomerId, and overall total quantity, utilizing the GROUPING SETS subclause.

In [60]:
-- Using the GROUPING SETS subclause
/*
USE TSQLV4

SELECT empid, custid, SUM(qty) AS sumqty
FROM dbo.Orders
GROUP BY
  GROUPING SETS
  (
    (empid, custid),
    (empid),
    (custid),
    ()
  );
*/
USE Northwinds2022TSQLV7
SELECT EmployeeId, CustomerId, SUM(Quantity) AS SumQuantity
FROM dbo.Orders
GROUP BY
  GROUPING SETS 
  (
    (EmployeeId, CustomerId),
    (EmployeeId),
    (CustomerId),
    ()
  );

(15 rows affected)

Total execution time: 00:00:00.004

EmployeeId,CustomerId,SumQuantity
2,A,52
3,A,20
NULL,A,72
1,B,20
2,B,27
NULL,B,47
1,C,34
3,C,22
NULL,C,56
3,D,30


**Explanation**

- The proposition is to generate a query against the dbo.Orders table that calculates the total quantity of products ordered, considering various grouping combinations such as by EmployeeId, CustomerId, both EmployeeId and CustomerId, and overall total quantity, utilizing the GROUPING SETS subclause. The tables used are dbo.orders containing columns EmployeeId, CustomerId, and Quantity. The predicate uses
    - SUM(Quantity) AS SumQuantity to calculate the total quantity of products ordered for each grouping combination specified in the GROUPING SETS clause. 
    - GROUP BY GROUPING SETS (...): The GROUPING SETS clause specifies the various grouping combinations for aggregating the data. Each grouping set within the clause defines a different combination of grouping columns, including (EmployeeId, CustomerId), (EmployeeId), (CustomerId), and an empty set (), representing the overall total quantity.
- This query provides a comprehensive analysis of product orders by different combinations of EmployeeId and CustomerId, enabling businesses to gain insights into sales performance, customer behavior, and employee productivity across various grouping dimensions.

## _TOP 3_

**Proposition**

- Write a query against the dbo.Orders table that calculates the total quantity of products ordered, considering all possible combinations of grouping by EmployeeId, CustomerId, and their combinations, utilizing the CUBE grouping operator

In [62]:
-- Using the CUBE subclause
/*
USE TSQLV4
SELECT empid, custid, SUM(qty) AS sumqty
FROM dbo.Orders
GROUP BY CUBE(empid, custid);
*/

USE Northwinds2022TSQLV7
SELECT EmployeeId, CustomerId, SUM(Quantity) AS SumQuantity
FROM dbo.Orders
GROUP BY CUBE(EmployeeId, CustomerId);


(15 rows affected)

Total execution time: 00:00:00.003

EmployeeId,CustomerId,SumQuantity
2,A,52
3,A,20
NULL,A,72
1,B,20
2,B,27
NULL,B,47
1,C,34
3,C,22
NULL,C,56
3,D,30


**Explanation**

- The proposition is to generate a query against the dbo.Orders table that calculates the total quantity of products ordered, considering all possible combinations of grouping by EmployeeId, CustomerId, and their combinations, utilizing the CUBE grouping operator. The table used was dbo.Orders using the columns EmployeeId, CustomerId, and Quantity. The predicate was:
    - SUM(Quantity) AS SumQuantity which calculates the total quantity of products ordered for each grouping combination generated by the CUBE grouping operator.
    - GROUP BY CUBE(EmployeeId, CustomerId) uses CUBE grouping operator generates all possible combinations of grouping columns specified in the GROUP BY clause. In this case, it considers all possible combinations of grouping by EmployeeId, CustomerId, and their combinations.
- This query provides a comprehensive analysis of product orders by considering all possible combinations of grouping by EmployeeId and CustomerId, allowing businesses to gain insights into sales performance, customer behavior, and employee productivity across various dimensions.

## _TOP 4_

**Proposition**

- Write a query against the dbo.Orders table that calculates the total quantity of products ordered, grouped by year, month, and day of the order date, while also providing subtotal quantities for each year, month, and day, utilizing the ROLLUP grouping operator.

In [85]:
-- Using the ROLLUP subclause
/*
USE TSQLV4
SELECT 
  YEAR(orderdate) AS orderyear,
  MONTH(orderdate) AS ordermonth,
  DAY(orderdate) AS orderday,
  SUM(qty) AS sumqty
FROM dbo.Orders
GROUP BY ROLLUP(YEAR(orderdate), MONTH(orderdate), DAY(orderdate));
*/

USE Northwinds2022TSQLV7
SELECT 
  YEAR(OrderDate) AS orderyear,
  MONTH(OrderDate) AS ordermonth,
  DAY(OrderDate) AS orderday,
  SUM(Quantity) AS sumqty
FROM dbo.Orders
GROUP BY ROLLUP(YEAR(OrderDate), MONTH(OrderDate), DAY(OrderDate));

(22 rows affected)

Total execution time: 00:00:00.014

orderyear,ordermonth,orderday,sumqty
2014,4,18,22
2014,4,NULL,22
2014,8,2,10
2014,8,NULL,10
2014,12,24,32
2014,12,NULL,32
2014,NULL,NULL,64
2015,1,9,40
2015,1,18,14
2015,1,NULL,54


**Explanation**

- The proposition is to write a query against the dbo.Orders table that calculates the total quantity of products ordered, grouped by year, month, and day of the order date. Additionally, the query should provide subtotal quantities for each year, month, and day, utilizing the ROLLUP grouping operator. Tables used are dbo.Orders and selected columns OrderDate and Quantity. The predicate was:

    - YEAR(OrderDate) AS orderyear: Extracts the year from the OrderDate column and aliases it as orderyear.
    - MONTH(OrderDate) AS ordermonth: Extracts the month from the OrderDate column and aliases it as ordermonth.
    - DAY(OrderDate) AS orderday: Extracts the day from the OrderDate column and aliases it as orderday.
    - SUM(Quantity) AS sumqty: Calculates the total quantity of products ordered for each grouping combination.
    - GROUP BY ROLLUP(YEAR(OrderDate), MONTH(OrderDate), DAY(OrderDate)): The ROLLUP grouping operator generates subtotal quantities for each combination of grouping columns specified in the GROUP BY clause. In this case, it generates subtotals for each year, month, and day of the order date.

- This query provides a detailed analysis of product orders by breaking down the total quantity ordered into subtotals for each year, month, and day. By using the ROLLUP operator, the query produces subtotal quantities for various levels of granularity, allowing for a comprehensive understanding of order patterns over time.

## _Top 5_

**Proposition**

- Write a query against the Northwinds2022TSQLV7 database that calculates the total quantity of products ordered, considering all possible combinations of grouping by EmployeeId, CustomerId, and their combinations, utilizing the CUBE grouping operator.

In [65]:
/*
USE TSQLV4
SELECT empid, custid, SUM(qty) AS sumqty
FROM dbo.Orders
GROUP BY CUBE(empid, custid);
*/

USE Northwinds2022TSQLV7
SELECT EmployeeId, CustomerId, SUM(Quantity) AS SumQuantity
FROM dbo.Orders
GROUP BY CUBE(EmployeeId, CustomerId);

(15 rows affected)

Total execution time: 00:00:00.004

EmployeeId,CustomerId,SumQuantity
2,A,52
3,A,20
NULL,A,72
1,B,20
2,B,27
NULL,B,47
1,C,34
3,C,22
NULL,C,56
3,D,30


**Explanation**

- The proposition is to write a query against the Orders table in the Northwinds2022TSQLV7 database. The query aims to calculate the total quantity of products ordered, considering all possible combinations of grouping by EmployeeId, CustomerId, and their combinations. This is achieved by utilizing the CUBE grouping operator. The table used is dbo.Orders where it selects the columns EmployeeId, CustomerId, and Quantity. The predicate is:

    - SUM(Quantity) AS SumQuantity: Calculates the total quantity of products ordered for each grouping combination.
    - GROUP BY CUBE(EmployeeId, CustomerId): The CUBE grouping operator generates all possible combinations of grouping columns specified in the GROUP BY clause. In this case, it considers all possible combinations of grouping by EmployeeId, CustomerId, and their combinations.
- This query provides a comprehensive analysis of product orders by considering all possible combinations of grouping by EmployeeId and CustomerId.The CUBE grouping operator generates subtotal quantities for various combinations of grouping columns, allowing for a detailed examination of product orders across different dimensions.

## _Q6._

**Proposition**

- Write a query against the dbo.Orders table that calculates the total quantity of products ordered, considering all possible combinations of grouping by EmployeeId, CustomerId, and their combinations. Additionally, the query should include indicators for grouping EmployeeId and CustomerId columns, utilizing the CUBE grouping operator

In [58]:
/*
USE TSQLV4
SELECT
  GROUPING(empid) AS grpemp,
  GROUPING(custid) AS grpcust,
  empid, custid, SUM(qty) AS sumqty
FROM dbo.Orders
GROUP BY CUBE(empid, custid);
*/

USE Northwinds2022TSQLV7
SELECT
  GROUPING(EmployeeId) AS GroupEmp,
  GROUPING(CustomerId) AS GroupCust,
  EmployeeId, CustomerId, SUM(Quantity) AS SumQuantity
FROM dbo.Orders
GROUP BY CUBE(EmployeeId, CustomerId);


(15 rows affected)

Total execution time: 00:00:00.007

GroupEmp,GroupCust,EmployeeId,CustomerId,SumQuantity
0,0,2,A,52
0,0,3,A,20
1,0,NULL,A,72
0,0,1,B,20
0,0,2,B,27
1,0,NULL,B,47
0,0,1,C,34
0,0,3,C,22
1,0,NULL,C,56
0,0,3,D,30


**Explanation**

- The proposition is to write a query against the dbo.Orders table that calculates the total quantity of products ordered, considering all possible combinations of grouping by EmployeeId, CustomerId, and their combinations. Additionally, the query should include indicators for grouping the EmployeeId and CustomerId columns, utilizing the CUBE grouping operator. The table used is dbo.Orders which selected EmployeeId, CustomerId, and Quantity columns. The predicate is:
    - SUM(Quantity) AS SumQuantity: Calculates the total quantity of products ordered for each grouping combination.
    - GROUP BY CUBE(EmployeeId, CustomerId): The CUBE grouping operator generates all possible combinations of grouping columns specified in the GROUP BY clause. In this case, it considers all possible combinations of grouping by EmployeeId, CustomerId, and their combinations.
- This query provides a comprehensive analysis of product orders by considering all possible combinations of grouping by EmployeeId and CustomerId. The inclusion of indicators (GroupEmp and GroupCust) for grouping the EmployeeId and CustomerId columns provides additional information about the grouping status of each column in the result set. By utilizing the CUBE grouping operator, the query generates subtotal quantities for various combinations of grouping columns, allowing for a detailed examination of product orders across different dimensions.

## _Q7._

**Proposition**

- Write a query against the dbo.Orders table that calculates the total quantity of products ordered, considering all possible combinations of grouping by EmployeeId, CustomerId, and their combinations. Additionally, the query should include a grouping identifier for each grouping set, utilizing the CUBE grouping operator.

In [57]:
/*
USE TSQLV4
SELECT
  GROUPING_ID(empid, custid) AS groupingset,
  empid, custid, SUM(qty) AS sumqty
FROM dbo.Orders
GROUP BY CUBE(empid, custid);
*/

USE Northwinds2022TSQLV7
SELECT
  GROUPING_ID(EmployeeId, CustomerId) AS groupingset,
  EmployeeId, CustomerId, SUM(Quantity) AS sumqty
FROM dbo.Orders
GROUP BY CUBE(EmployeeId, CustomerId);

(15 rows affected)

Total execution time: 00:00:00.008

groupingset,EmployeeId,CustomerId,sumqty
0,2,A,52
0,3,A,20
2,NULL,A,72
0,1,B,20
0,2,B,27
2,NULL,B,47
0,1,C,34
0,3,C,22
2,NULL,C,56
0,3,D,30


**Explanation**

- The proposition is to write a query against the dbo.Orders table that calculates the total quantity of products ordered, considering all possible combinations of grouping by EmployeeId, CustomerId, and their combinations. Additionally, the query should include a grouping identifier for each grouping set, utilizing the CUBE grouping operator. The table used is dbo.Orders selecting EmployeeId, CustomerId, and Quantity. The predicate is:
    - SUM(Quantity) AS sumqty: Calculates the total quantity of products ordered for each grouping combination.
    - GROUP BY CUBE(EmployeeId, CustomerId): The CUBE grouping operator generates all possible combinations of grouping columns specified in the GROUP BY clause. In this case, it considers all possible combinations of grouping by EmployeeId, CustomerId, and their combinations.

- This query provides a comprehensive analysis of product orders by considering all possible combinations of grouping by EmployeeId and CustomerId. The inclusion of a grouping identifier (groupingset) for each grouping set allows for easy identification and analysis of different grouping combinations. By utilizing the CUBE grouping operator, the query generates subtotal quantities for various combinations of grouping columns, enabling a detailed examination of product orders across different dimensions.

In [ ]:
USE TSQLV4
DROP TABLE IF EXISTS dbo.Orders;

USE Northwinds2022TSQLV7
DROP TABLE IF EXISTS dbo.Orders;